In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd '/gdrive/My Drive/Colab Notebooks/ECE_5424_Intelligent_Autonomous_Systems/Project_4'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive/My Drive/Colab Notebooks/ECE_5424_Intelligent_Autonomous_Systems/Project_4


In [0]:
from maze import *
import numpy as np

In [0]:
def update_policy(weights, Q_vals, dcnt, maze):
    diff = 1e-200 + 1
    while True:
        diff = 0
        # Iterate through all states and actions to find the weighted values
        # with the discount factor for the action with the highest value
        for state in range (112):
            val = weights[state]
            next_valz = np.zeros(4)
            for act in range(4):           
                reward, next_state, done = maze.step(state, act)
                next_valz[act] = dcnt * (dcnt * weights[next_state] + reward)
                Q_vals[state, act] = next_valz[act]      
                reward_slip, nxt_s_slip, done = maze.step(state, ACTMAP[act])
                next_valz[act] = next_valz[act] +  (1 - dcnt) * (dcnt * weights[nxt_s_slip] + reward_slip)
            weights[state] = np.max(next_valz)
            
            diff = max(diff, np.abs(val - weights[state]))
        if (diff < 1e-200):
            break
            
    return weights

In [0]:
# Initialize Values
maze = Maze()
ACTMAP = {0:3, 1:2, 2:0, 3:1}
dcnt = 0.9 # Discount Factor

actions = np.random.choice(4, 112)
weights = np.zeros(112)
maze.slip = 0 # For finding policy, so slip = 0
Q_vals = np.zeros((112, 4))

P_conv = False

In [0]:
# Keep updating the policy until the optimal policy is met
# This is checked by checking if the new policy is different from the previous one
# If it isn't different, we exit the while loop
while (P_conv == False):
    weights = update_policy(weights, Q_vals, dcnt, maze)
    P_conv = True
    for state in range(112):
        prev_act = actions[state]
        next_vals = np.zeros(112)        
        
        for act in range(4):
            reward, next_state, done = maze.step(state,act)
            next_vals[act] = weights[next_state]
        actions[state] = np.argmax(next_vals)
        if prev_act != actions[state]:
            P_conv = False

In [7]:
# Save and print the Q table
np.save('Q_table', Q_vals)
print(Q_vals)

[[ 4.28644758  4.81563864  4.28644758  4.28644758]
 [ 5.57408958  6.26224879  5.57408958  5.57408958]
 [ 6.99115255  7.85425781  6.99115255  6.99115255]
 [ 6.80647131  7.64677641  6.80647131  6.80647131]
 [ 9.02102589 10.13473279  9.02102589  9.02102589]
 [ 8.69471111  9.76813224  8.69471111  8.69471111]
 [10.27902583 11.54804136 10.27902583 10.27902583]
 [13.0287919  14.63728473 13.0287919  13.0287919 ]
 [ 4.28644758  4.81563864  4.81563864  5.41016193]
 [ 5.57408958  6.26224879  6.26224879  7.03536592]
 [ 6.99115255  7.85425781  7.85425781  8.82391927]
 [ 6.80647131  7.64677641  7.64677641  8.59082288]
 [ 9.02102589 10.13473279 10.13473279 11.38593437]
 [ 8.69471111  9.76813224  9.76813224 10.97407449]
 [10.27902583 11.54804136 11.54804136 12.97372548]
 [13.0287919  14.63728473 14.63728473 16.44435691]
 [ 5.67384556  6.30427284  6.30427284  5.55314604]
 [ 8.25116441  9.16796045  9.16796045  8.13056994]
 [ 6.30427284  6.30427284  6.30427284  7.08257813]
 [ 8.33106032  9.25673369  9.25

In [87]:
np.save('Policy', actions)
print(actions)

[1 1 1 1 1 1 1 1 3 3 3 3 3 3 3 3 1 1 3 1 3 1 3 3 3 1 3 3 3 3 3 3 1 3 3 3 3
 3 3 3 2 2 0 2 0 2 0 0 0 1 0 0 1 1 0 1 0 1 0 0 3 3 0 3 0 3 0 0 3 3 0 3 2 1
 2 2 3 3 3 3 1 1 1 0 1 0 0 0 1 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 2 2 2 2 0 0 0
 0]


In [85]:
# Reset the maze to show the optimal path
initial_state = maze.reset()
state = initial_state
done = False
while(done == False):
    action = actions[state]
    reward, state, done = maze.step(state, action)
    position = np.floor(state/8) * 8
    flags = state - position
    maze.plot(state, action)

action:  DOWN
SWFWG
OOOOO
WOOOW
FOWFW
action:  RIGHT
SWFWG
OOOOO
WOOOW
FOWFW
action:  RIGHT
SWFWG
OOOOO
WOOOW
FOWFW
action:  UP
SWFWG
OOOOO
WOOOW
FOWFW
action:  DOWN
SWFWG
OOOOO
WOOOW
FOWFW
action:  DOWN
SWFWG
OOOOO
WOOOW
FOWFW
action:  RIGHT
SWFWG
OOOOO
WOOOW
FOWFW
action:  DOWN
SWFWG
OOOOO
WOOOW
FOWFW
action:  UP
SWFWG
OOOOO
WOOOW
FOWFW
action:  UP
SWFWG
OOOOO
WOOOW
FOWFW
action:  RIGHT
SWFWG
OOOOO
WOOOW
FOWFW
action:  UP
SWFWG
OOOOO
WOOOW
FOWFW
